In [2]:
import pandas as pd

## Fetching Tweet data from Twitter

In [ ]:
politifact_fake_df = pd.read_csv("data/FakeNewsNet/dataset/politifact_fake.csv")

pf_fake_tweet_ids = politifact_fake_df['tweet_ids'].str.split('\t').explode().tolist()
print(len(pf_fake_tweet_ids))
print()

politifact_real_df = pd.read_csv("data/FakeNewsNet/dataset/politifact_real.csv")
pf_real_tweet_ids = politifact_real_df['tweet_ids'].str.split('\t').explode().tolist()
print(len(pf_real_tweet_ids))

# def concat_tweet_ids(row):
#     global pf_fake_tweet_ids
#     print('tweet_ids' in row)
#     if 'tweet_ids' in row and row['tweet_ids'] != "":
#         pf_fake_tweet_ids = pf_fake_tweet_ids + row['tweet_ids'].split("\t")
    
# politifact_fake_df.apply(lambda x: concat_tweet_ids(x))
# print(pf_fake_tweet_ids)

fake_short = pf_fake_tweet_ids[:20]

print(fake_short)

In [ ]:
api_key = os.environ.get('APIKey')
api_key_secret = os.environ.get('APIKeySecret')
access_token = os.environ.get('AccessToken')
access_token_secret = os.environ.get('AccessTokenSecret')
bearer_token = os.environ.get('BearerToken')

In [ ]:
# Remove nans (don't kill grandmas though)
# Fake
print(len(pf_fake_tweet_ids))
print(np.nan in pf_fake_tweet_ids)
pf_fake_tweet_ids = [x for x in pf_fake_tweet_ids if x is not np.nan]
print(len(pf_fake_tweet_ids))
print(np.nan in pf_fake_tweet_ids)

# Real
print(len(pf_real_tweet_ids))
print(np.nan in pf_real_tweet_ids)
pf_real_tweet_ids = [x for x in pf_real_tweet_ids if x is not np.nan]
print(len(pf_real_tweet_ids))
print(np.nan in pf_real_tweet_ids)

In [ ]:
classes = [pf_fake_tweet_ids, pf_real_tweet_ids]
for news_class in classes:
    IDs = news_class
    n = len(IDs)
    for batch_ix in range(0, n, 100):
        if batch_ix % 500 == 0:
            print("Tweets analysed so far:", batch_ix)

        response = client.get_tweets(IDs[batch_ix : min(batch_ix + 100, n)], 
                                     tweet_fields=
                                         ["created_at", "text", "lang", "author_id", "source"],
                                     user_fields=
                                         ["location", "description", "verified"],
                                     expansions="author_id")

        # https://www.kirenz.com/post/2021-12-10-twitter-api-v2-tweepy-and-pandas-in-python/twitter-api-v2-tweepy-and-pandas-in-python/
        # Save data as dictionary
        tweets_dict = response.json() 

        # Extract "data" value from dictionary
        try:
            tweets_data = tweets_dict['data'] # can fail if nans in Tweets
            user_data = tweets_dict['includes']['users']

            # Transform to pandas Dataframe
            df_tweets = pd.json_normalize(tweets_data)
            df_tweets.head()

            # Transform to pandas Dataframe
            df_users = pd.json_normalize(user_data)
            df_users = df_users.rename(columns={'id': 'author_id'})
            df_users.head()

            df = pd.merge(df_tweets, df_users, on=["author_id"])

            # Standardise order of columns
            df = df[['id', 'text', 'author_id', 'source', 'created_at', 'edit_history_tweet_ids', 'lang', 'description', 'username', 'verified', 'name', 'location']]

            include_header = batch_ix == 0
            df.to_csv("real.csv", mode='a', header=include_header)
        except:
            continue

## Connecting original and later-fetched real news

In [4]:
og = pd.read_csv("data/FakeNewsNet/real.csv")
later = pd.read_csv("real.csv")

# frames = [og, later]
# new = pd.concat(frames)

# new.to_csv("real_new.csv")

new = pd.read_csv("real_new.csv")

117009
179162
296171


In [6]:
print(len(og))
print(len(later))
print(len(new))
print(len(og) + len(later) == len(new))

117009
179162
296171
True
